*Step.1 導入相關套件*

In [ ]:
!pip install datasets

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

*Step.2 載入數據*

In [ ]:
ds = Dataset.load_from_disk("./alpaca_data_zh/")
ds

In [ ]:
ds[:3]

*Step.3 數據前處理*

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-389m-zh")
tokenizer

In [ ]:
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [ ]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

In [ ]:
tokenizer.decode(tokenized_ds[1]["input_ids"])

In [ ]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

*Step.4 模型建立*

In [ ]:
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-389m-zh")

*Step.5 訓練參數*

In [ ]:
args = TrainingArguments(
    output_dir = "./chatbot",
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 8,
    logging_steps = 10,
    num_train_epochs = 2
)

*Step.6 訓練器*

In [ ]:
trainer = Trainer(
    model = model,
    args = args,
    tokenizer = tokenizer,
    train_dataset = tokenized_ds,
    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
)

*Step.7 模型訓練*

In [ ]:
trainer.train()

*Step.8 模型推理*

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

In [ ]:
ipt = "Human: {}\n{}".format("考試有哪些技巧?", "").strip() + "\n\nAssistant: "
pipe(ipt, max_length=256, do_sample=True, )